# Making predictions using streaming aggregated features

All prior notebooks have been setting up our end to end solution. Now that all those steps are complete, it is time to see the solution in action. In this notebook, we will send credit card transactions to our input Kinesis stream and show that we can detect fraud. We take advantage of multiple online feature groups in the Amazon SageMaker Feature Store. One of those feature groups is refreshed by a processing job, which we would run nightly to provide aggregate features looking back one week. The other feature group uses streaming ingestion to aggregate features that look back over a rolling 10-minute window.

<img src="./images/streaming_prediction.png" />


### Recap of what is in place

Here is a recap of what we have done so far:

1. In [notebook 0](./0_prepare_transactions_dataset.ipynb), We generated a synthetic dataset of transactions, including simulated fraud attacks.
2. In [notebook 1](./1_setup.ipynb), we created our two feature groups. In that same notebook, we also created a Kinesis data stream and a Kinesis Data Analytics SQL application that consumes the transaction stream and produces aggregate features. These features are provided in near real time to Lambda, and they look back over a 10 minute window.
3. In [notebook 2](./2_batch_ingestion.ipynb), we used a SageMaker Processing Job to create aggregated features and used them to feed both the training dataset as well as an online feature group.
4. In [notebook 3](./3_train_and_deploy_model.ipynb), we trained and deployed an XGBoost model to detect fraud.
5. Our [CloudFormation template](https://console.aws.amazon.com/cloudformation/home) deployed a pair of Lambda functions. One listens to the KDA SQL output and keeps the `cc-agg-fg` feature group up to date. The other Lambda listens to the Kinesis data stream for transactions, pulls a set of features from multiple feature groups, and invokes our fraud detection endpoint, as seen in the above picture.

## Imports and overall setup

### Imports and initialization

In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import json
import time

### Access the transaction test dataset

In [ ]:
test_file_path = './data/test.csv'
test_df = pd.read_csv(test_file_path)
test_df.head()

In [ ]:
test_df.head()
print(test_df.shape)

In [ ]:
def put_to_stream(stream_name, cc_num, merchant, amount, zip_code, timestamp):
    
    payload = {
        'cc_num': int(cc_num),
        'merchant': merchant,
        'amount': amount,
        'zip_code': zip_code,
        'trans_ts': timestamp
    }
    ret_status = True
    data = json.dumps(payload)
    print(f'Sending transaction on card: {cc_num}...')
    response = kinesis_client.put_record(StreamName = stream_name,
                                             Data = data,
                                             PartitionKey = 'shard1')
    
    if (response['ResponseMetadata']['HTTPStatusCode'] != 200):
        print("ERROR: Kinesis put_record failed: \n{}".format(json.dumps(response)))
        ret_status = False
        
    return ret_status

In [ ]:
def simulate_fraud(stream_name, cc_num):
    min_wait = 1; max_wait = 2
    for i in range(10):
        random_amt = round(np.random.uniform(1.00, 50.00), 2)
        seconds_to_wait = np.random.uniform(min_wait, max_wait)
        print(f'waiting {seconds_to_wait:.1f} seconds to send trans {i}...')
        time.sleep(seconds_to_wait)
        put_to_stream(stream_name, int(cc_num), 'Random Corp', random_amt, '03099', time.time())

### Send some transactions, and see the results
To show that the solution works, we send a single transaction to each of three different credit cards. Then, we simulate a fraud attack on a 4th credit card by sending many transactions in quick succession. The output from our Lambda function is then shown from CloudWatch log streams. Here's an example of what you should see as a result:

<img src="./images/log_results.png" />

As expected, the first three one-off transactions are predicted as NOT FRAUD. Of the ten fraudulent transactions, the first is predicted as NOT FRAUD, and the rest are all correctly identified as FRAUD. Notice how the aggregate features are kept current, helping drive more accurate predictions.

Now let's give it a shot.

In [ ]:
cc_nums = test_df.cc_num.unique()[10:14]

start_test_time = time.time() 

put_to_stream(STREAM_NAME, cc_nums[0], 'Merchant-0', round(np.random.uniform(100, 5000), 2), 'zip-0', time.time())
put_to_stream(STREAM_NAME, cc_nums[1], 'Merchant-1', round(np.random.uniform(100, 5000), 2), 'zip-1', time.time())
put_to_stream(STREAM_NAME, cc_nums[2], 'Merchant-2', round(np.random.uniform(100, 5000), 2), 'zip-2', time.time())

print('\nNow simulate a fraud attack...')
fraud_cc_num = cc_nums[3]
simulate_fraud(STREAM_NAME, fraud_cc_num)

end_test_time = time.time() 

### Feed a stream of transactions [optional]
If you would like to send additional credit card transactions to simulate more input traffic to the feature pipeline, you can pull from the test dataset as shown below. Just pass in how many transactions you want to send, and the max wait time between transactions (in seconds).

In [ ]:
import time

def simulate_traffic(df, max_wait, num_trans):
    for i in range(num_trans):
        row = test_df.iloc[i]
        cc_num = row['cc_num']
        zip_code = '0'
        merchant = 'A'
        amt = row['amount']
        print(f'cc_num: {cc_num}, amt: {amt}')
        seconds_to_wait = int(np.random.uniform(0.1, max_wait))
        print(f'waiting {seconds_to_wait} seconds to send trans {i}...')
        time.sleep(seconds_to_wait)
        print(f' putting trans with card: {cc_num}, amt: {amt}, zip: {zip_code}, merchant: {merchant}')
        status = put_to_stream(STREAM_NAME,cc_num, merchant, amt, zip_code, time.time())
        if (not status):
            print('Error found during write to Kinesis Stream')
            break
        i += 1
        if i > (num_trans -1):
            break

In [ ]:
#simulate_traffic(test_df, 2, 5)